In [1]:
import pandas as pd
import numpy as np
import joblib

# --- 1. Load main auction data ---
main_df = pd.read_excel('../data/AuctionData.xlsx')
main_df.columns = main_df.columns.str.lower()
main_df['auction_date'] = pd.to_datetime(main_df['auction_date'])

# --- 2. Filter for cylinders and date range ---
# Adjust this filter as per your actual product_group logic
if 'product_group' not in main_df.columns:
    # Example: infer from productdescription
    cylinders = [x.lower().strip() for x in [
        '14.2 Kg', '19 Kg', '5 Kg', '47.5 Kg', '5 Kg FTLR', '5 Kg ND',
        '47.5 Kg LOTV', '19 Kg SC', '19 Kg NCut', '5 Kg FTL', '14.2 Kg OMC'
    ]]
    def map_product_type(desc):
        desc = str(desc).lower().strip()
        return 'cylinders' if desc in cylinders else 'other'
    main_df['product_group'] = main_df['productdescription'].apply(map_product_type)

cyl_df = main_df[
    (main_df['auction_date'] >= '2025-01-01') &
    (main_df['auction_date'] <= '2025-07-07') &
    (main_df['product_group'] == 'cylinders')
].copy()

# --- 3. Feature engineering ---
cyl_df['year'] = cyl_df['auction_date'].dt.year
cyl_df['month'] = cyl_df['auction_date'].dt.month

# For EWM features, use the last value from training (simulate production)
# You should replace these with the actual last values from your training set
last_ewm_proposed_rp = 35000  # <-- replace with your value
last_ewm_last_bid_price = 34000  # <-- replace with your value
cyl_df['ewm_proposed_rp'] = last_ewm_proposed_rp
cyl_df['ewm_last_bid_price'] = last_ewm_last_bid_price

# --- 4. Predict using the trained models ---
cyl_features = ['year', 'month', 'quantity', 'ewm_proposed_rp', 'ewm_last_bid_price']
model_q25 = joblib.load('../models/cyl_proposed_rp_q25.joblib')
model_q50 = joblib.load('../models/cyl_proposed_rp_q50.joblib')
model_q75 = joblib.load('../models/cyl_proposed_rp_q75.joblib')

X_pred = cyl_df[cyl_features]
cyl_df['pred_q25'] = model_q25.predict(X_pred)
cyl_df['pred_q50'] = model_q50.predict(X_pred)
cyl_df['pred_q75'] = model_q75.predict(X_pred)

# --- 5. Compare predictions to actuals ---
cyl_df['actual'] = cyl_df['proposed_rp']
cyl_df['abs_error'] = (cyl_df['pred_q50'] - cyl_df['actual']).abs()
cyl_df['pct_error'] = 100 * cyl_df['abs_error'] / cyl_df['actual']
cyl_df['in_range'] = cyl_df.apply(
    lambda row: min(row['pred_q25'], row['pred_q75']) <= row['actual'] <= max(row['pred_q25'], row['pred_q75']),
    axis=1
)

# --- 6. Print summary metrics ---
mae = cyl_df['abs_error'].mean()
rmse = np.sqrt(((cyl_df['pred_q50'] - cyl_df['actual']) ** 2).mean())
coverage = cyl_df['in_range'].mean() * 100

print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"Coverage (actual in Q25-Q75): {coverage:.1f}%")

# --- 7. Save results to Excel ---
cyl_df[['auction_date', 'quantity', 'actual', 'pred_q25', 'pred_q50', 'pred_q75', 'abs_error', 'pct_error', 'in_range']].to_excel('cylinder_rp_eval_results.xlsx', index=False)
print("Results saved to cylinder_rp_eval_results.xlsx") 

MAE: 3320.59
RMSE: 3670.96
Coverage (actual in Q25-Q75): 13.1%
Results saved to cylinder_rp_eval_results.xlsx


In [2]:
import pandas as pd
import numpy as np
import joblib

# --- 1. Load main auction data ---
main_df = pd.read_excel('../data/AuctionData.xlsx')
main_df.columns = main_df.columns.str.lower()
main_df['auction_date'] = pd.to_datetime(main_df['auction_date'])

# --- 2. Filter for cylinders and date range ---
if 'product_group' not in main_df.columns:
    cylinders = [x.lower().strip() for x in [
        '14.2 Kg', '19 Kg', '5 Kg', '47.5 Kg', '5 Kg FTLR', '5 Kg ND',
        '47.5 Kg LOTV', '19 Kg SC', '19 Kg NCut', '5 Kg FTL', '14.2 Kg OMC'
    ]]
    def map_product_type(desc):
        desc = str(desc).lower().strip()
        return 'cylinders' if desc in cylinders else 'other'
    main_df['product_group'] = main_df['productdescription'].apply(map_product_type)

cyl_df = main_df[
    (main_df['auction_date'] >= '2025-01-01') &
    (main_df['auction_date'] <= '2025-07-07') &
    (main_df['product_group'] == 'cylinders')
].copy()

# --- 3. Feature engineering ---
cyl_df['year'] = cyl_df['auction_date'].dt.year
cyl_df['month'] = cyl_df['auction_date'].dt.month

# For EWM features, use the last value from training (simulate production)
# You should replace these with the actual last values from your training set
last_ewm_proposed_rp = 35000  # <-- replace with your value
last_ewm_last_bid_price = 34000  # <-- replace with your value
cyl_df['ewm_proposed_rp'] = last_ewm_proposed_rp
cyl_df['ewm_last_bid_price'] = last_ewm_last_bid_price

# --- 4. Predict using the trained models ---
cyl_features = ['year', 'month', 'quantity', 'ewm_proposed_rp', 'ewm_last_bid_price']
model_q25 = joblib.load('../models/cyl_lbp_q25.joblib')
model_q50 = joblib.load('../models/cyl_lbp_q50.joblib')
model_q75 = joblib.load('../models/cyl_lbp_q75.joblib')

X_pred = cyl_df[cyl_features]
cyl_df['pred_q25'] = model_q25.predict(X_pred)
cyl_df['pred_q50'] = model_q50.predict(X_pred)
cyl_df['pred_q75'] = model_q75.predict(X_pred)

# --- 5. Compare predictions to actuals ---
cyl_df['actual'] = cyl_df['last_bid_price']
cyl_df['abs_error'] = (cyl_df['pred_q50'] - cyl_df['actual']).abs()
cyl_df['pct_error'] = 100 * cyl_df['abs_error'] / cyl_df['actual']
cyl_df['in_range'] = cyl_df.apply(
    lambda row: min(row['pred_q25'], row['pred_q75']) <= row['actual'] <= max(row['pred_q25'], row['pred_q75']),
    axis=1
)

# --- 6. Print summary metrics ---
mae = cyl_df['abs_error'].mean()
rmse = np.sqrt(((cyl_df['pred_q50'] - cyl_df['actual']) ** 2).mean())
coverage = cyl_df['in_range'].mean() * 100

print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"Coverage (actual in Q25-Q75): {coverage:.1f}%")

# --- 7. Save results to Excel ---
cyl_df[['auction_date', 'quantity', 'actual', 'pred_q25', 'pred_q50', 'pred_q75', 'abs_error', 'pct_error', 'in_range']].to_excel('cylinder_lbp_eval_results.xlsx', index=False)
print("Results saved to cylinder_lbp_eval_results.xlsx") 

MAE: 11497.01
RMSE: 11669.23
Coverage (actual in Q25-Q75): 0.0%
Results saved to cylinder_lbp_eval_results.xlsx


d:\python3.9\lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but HistGradientBoostingRegressor was fitted without feature names
  warnings.warn(
d:\python3.9\lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but HistGradientBoostingRegressor was fitted without feature names
  warnings.warn(
d:\python3.9\lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but HistGradientBoostingRegressor was fitted without feature names
  warnings.warn(


In [4]:

cu_price = {
    '2025-07-07': 891.9000244140625,
    '2025-07-04': 898.8499755859375,
    '2025-07-03': 905.4000244140625,
    '2025-07-02': 898.2999877929688,
    '2025-07-01': 894.5999755859375,
    '2025-06-30': 892.9500122070312,
    '2025-06-27': 896.0,
    '2025-06-26': 885.75,
    '2025-06-25': 881.3499755859375,
    '2025-06-24': 887.9000244140625,
    '2025-06-23': 886.3499755859375,
    '2025-06-20': 884.25,
    '2025-06-19': 886.0499877929688,
    '2025-06-18': 885.4500122070312,
    '2025-06-17': 883.7000122070312,
    '2025-06-16': 880.5499877929688,
    '2025-06-13': 879.25,
    '2025-06-12': 876.0499877929688,
    '2025-06-11': 883.8499755859375,
    '2025-06-10': 886.5999755859375,
    '2025-06-09': 879.57,
    '2025-06-06': 883.9500122070312,
    '2025-06-05': 878.5999755859375,
    '2025-06-04': 874.5499877929688,
    '2025-06-03': 871.6500244140625,
    '2025-06-02': 864.4000244140625,
    '2025-05-30': 866.2000122070312,
    '2025-05-29': 866.1500244140625,
    '2025-05-28': 867.0,
    '2025-05-27': 868.6500244140625,
    '2025-05-26': 870.5,
    '2025-05-23': 866.5999755859375,
    '2025-05-13': 856.5499877929688,
    '2025-05-12': 857.5,
    '2025-05-09': 857.0499877929688,
    '2025-05-08': 851.0499877929688,
    '2025-05-07': 853.4500122070312,
    '2025-05-06': 849.2000122070312,
    '2025-05-05': 844.1500244140625,
    '2025-05-02': 833.6500244140625,
    '2025-05-01': 830.0499877929688,
    '2025-04-30': 859.7000122070312,
    '2025-04-29': 853.4500122070312,
    '2025-04-28': 858.7999877929688,
    '2025-04-25': 866.1500244140625
}


In [5]:
zn_price = {
    '2025-07-07': 256.70001220703125,
    '2025-07-04': 257.45001220703125,
    '2025-07-03': 258.45001220703125,
    '2025-07-02': 256.0,
    '2025-07-01': 258.3500061035156,
    '2025-06-30': 259.8500061035156,
    '2025-06-27': 260.0,
    '2025-06-26': 256.29998779296875,
    '2025-06-25': 254.14999389648438,
    '2025-06-24': 255.6999969482422,
    '2025-06-23': 253.3000030517578,
    '2025-06-20': 253.6999969482422,
    '2025-06-19': 252.85000610351562,
    '2025-06-18': 252.5,
    '2025-06-17': 252.75,
    '2025-06-16': 250.1999969482422,
    '2025-06-13': 250.14999389648438,
    '2025-06-12': 250.9499969482422,
    '2025-06-11': 251.39999389648438,
    '2025-06-10': 252.10000610351562,
    '2025-06-09': 253.1999969482422,
    '2025-06-06': 255.6999969482422,
    '2025-06-05': 256.6000061035156,
    '2025-06-04': 256.20001220703125,
    '2025-06-03': 255.64999389648438,
    '2025-06-02': 252.35000610351562,
    '2025-05-30': 255.35000610351562,
    '2025-05-29': 255.9499969482422,
    '2025-05-28': 257.70001220703125,
    '2025-05-27': 257.75,
    '2025-05-26': 256.6000061035156,
    '2025-05-23': 258.0,
    '2025-05-13': 254.14999389648438,
    '2025-05-12': 250.85000610351562,
    '2025-05-09': 250.89999389648438,
    '2025-05-08': 248.0500030517578,
    '2025-05-07': 249.75,
    '2025-05-06': 248.75,
    '2025-05-05': 248.64999389648438,
    '2025-05-02': 247.60000610351562,
    '2025-05-01': 249.35000610351562,
    '2025-04-30': 253.1999969482422,
    '2025-04-29': 254.39999389648438,
    '2025-04-28': 259.3500061035156,
    '2025-04-25': 249.10000610351562,
    '2025-04-21': 249.10000610351562
}

In [9]:
cu_df = pd.DataFrame(list(cu_price.items()), columns=['date', 'cu_price'])
cu_df['date'] = pd.to_datetime(cu_df['date'])
zn_df = pd.DataFrame(list(zn_price.items()), columns=['date', 'zn_price'])
zn_df['date'] = pd.to_datetime(zn_df['date'])

In [10]:
valve_df = valve_df.sort_values('auction_date')
cu_df = cu_df.sort_values('date')
zn_df = zn_df.sort_values('date')

In [11]:
valve_df = pd.merge_asof(
    valve_df, cu_df, left_on='auction_date', right_on='date', direction='backward'
).drop(columns=['date'])

# Merge zinc prices
valve_df = pd.merge_asof(
    valve_df, zn_df, left_on='auction_date', right_on='date', direction='backward'
).drop(columns=['date'])

In [14]:
import pandas as pd
import numpy as np
import joblib

# --- 1. Load main auction data ---
main_df = pd.read_excel('../data/AuctionData.xlsx')
main_df.columns = main_df.columns.str.lower()
main_df['auction_date'] = pd.to_datetime(main_df['auction_date'])

# --- 2. Filter for valves and date range ---
if 'product_group' not in main_df.columns:
    valves = [x.lower().strip() for x in ['SC Valve', 'LIQUID OFFTAKE VALVE']]
    def map_product_type(desc):
        desc = str(desc).lower().strip()
        return 'valves' if desc in valves else 'other'
    main_df['product_group'] = main_df['productdescription'].apply(map_product_type)

valve_df = main_df[
    (main_df['auction_date'] >= '2025-01-01') &
    (main_df['auction_date'] <= '2025-07-07') &
    (main_df['product_group'] == 'valves')
].copy()

# --- 3. Feature engineering ---
valve_df['year'] = valve_df['auction_date'].dt.year
valve_df['month'] = valve_df['auction_date'].dt.month

# For EWM features, use the last value from training (simulate production)
last_ewm_proposed_rp = 35000  # <-- replace with your value
last_ewm_last_bid_price = 34000  # <-- replace with your value
valve_df['ewm_proposed_rp'] = last_ewm_proposed_rp
valve_df['ewm_last_bid_price'] = last_ewm_last_bid_price

# --- 4. Map copper and zinc prices using merge_asof ---
# Assume cu_price and zn_price are defined as dictionaries
cu_df = pd.DataFrame(list(cu_price.items()), columns=['date', 'cu_price'])
cu_df['date'] = pd.to_datetime(cu_df['date'])
zn_df = pd.DataFrame(list(zn_price.items()), columns=['date', 'zn_price'])
zn_df['date'] = pd.to_datetime(zn_df['date'])

valve_df = valve_df.sort_values('auction_date')
cu_df = cu_df.sort_values('date')
zn_df = zn_df.sort_values('date')

valve_df = pd.merge_asof(
    valve_df, cu_df, left_on='auction_date', right_on='date', direction='backward'
).drop(columns=['date'])
valve_df = pd.merge_asof(
    valve_df, zn_df, left_on='auction_date', right_on='date', direction='backward'
).drop(columns=['date'])

# --- 5. Predict brass_index_poly using saved model ---
brass_index_pipeline = joblib.load('../models/brass_index_model.joblib')
valve_df_renamed = valve_df.rename(columns={
    'cu_price': 'spot price(rs.)_copper',
    'zn_price': 'spot price(rs.)_zinc'
})

# Drop any rows with NaN copper or zinc prices (should be rare after merge_asof)
valve_df_renamed = valve_df_renamed.dropna(subset=['spot price(rs.)_copper', 'spot price(rs.)_zinc'])
valve_df = valve_df.loc[valve_df_renamed.index]

valve_df['brass_index_poly'] = brass_index_pipeline.predict(
    valve_df_renamed[['spot price(rs.)_copper', 'spot price(rs.)_zinc']]
)

# --- 6. Predict using the trained models ---
valve_features = ['year', 'month', 'quantity', 'ewm_proposed_rp', 'ewm_last_bid_price', 'brass_index_poly']
model_q25 = joblib.load('../models/valve_lbp_q25.joblib')
model_q50 = joblib.load('../models/valve_lbp_q50.joblib')
model_q75 = joblib.load('../models/valve_lbp_q75.joblib')

X_pred = valve_df[valve_features]
valve_df['pred_q25'] = model_q25.predict(X_pred)
valve_df['pred_q50'] = model_q50.predict(X_pred)
valve_df['pred_q75'] = model_q75.predict(X_pred)

# --- 7. Compare predictions to actuals ---
valve_df['actual'] = valve_df['last_bid_price']
valve_df['abs_error'] = (valve_df['pred_q50'] - valve_df['actual']).abs()
valve_df['pct_error'] = 100 * valve_df['abs_error'] / valve_df['actual']
valve_df['in_range'] = valve_df.apply(
    lambda row: min(row['pred_q25'], row['pred_q75']) <= row['actual'] <= max(row['pred_q25'], row['pred_q75']),
    axis=1
)

# --- 8. Print summary metrics ---
mae = valve_df['abs_error'].mean()
rmse = np.sqrt(((valve_df['pred_q50'] - valve_df['actual']) ** 2).mean())
coverage = valve_df['in_range'].mean() * 100

print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"Coverage (actual in Q25-Q75): {coverage:.1f}%")

# --- 9. Save results to Excel ---
valve_df[['auction_date', 'quantity', 'actual', 'pred_q25', 'pred_q50', 'pred_q75', 'abs_error', 'pct_error', 'in_range']].to_excel('valve_lbp_eval_results.xlsx', index=False)
print("Results saved to valve_lbp_eval_results.xlsx") 

MAE: 10279.30
RMSE: 11924.65
Coverage (actual in Q25-Q75): 0.0%
Results saved to valve_lbp_eval_results.xlsx


d:\python3.9\lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but HistGradientBoostingRegressor was fitted without feature names
  warnings.warn(
d:\python3.9\lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but HistGradientBoostingRegressor was fitted without feature names
  warnings.warn(
d:\python3.9\lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but HistGradientBoostingRegressor was fitted without feature names
  warnings.warn(


In [19]:
model = joblib.load('../models/valve_proposed_rp_q50.joblib')
print(model.feature_name_)

['month', 'year', 'rp_per_qty', 'amt_per_qty', 'quantity_clipped', 'last_bid_price_clipped', 'brass_index_poly']


In [20]:
import pandas as pd
import numpy as np
import joblib

# --- 1. Load main auction data ---
main_df = pd.read_excel('../data/AuctionData.xlsx')
main_df.columns = main_df.columns.str.lower()
main_df['auction_date'] = pd.to_datetime(main_df['auction_date'])

# --- 2. Filter for valves and date range ---
if 'product_group' not in main_df.columns:
    valves = [x.lower().strip() for x in ['SC Valve', 'LIQUID OFFTAKE VALVE']]
    def map_product_type(desc):
        desc = str(desc).lower().strip()
        return 'valves' if desc in valves else 'other'
    main_df['product_group'] = main_df['productdescription'].apply(map_product_type)

valve_df = main_df[
    (main_df['auction_date'] >= '2025-01-01') &
    (main_df['auction_date'] <= '2025-07-07') &
    (main_df['product_group'] == 'valves')
].copy()

# --- 3. Feature engineering ---
valve_df['year'] = valve_df['auction_date'].dt.year
valve_df['month'] = valve_df['auction_date'].dt.month
# Feature engineering for valve_df
valve_df['rp_per_qty'] = valve_df['proposed_rp'] / valve_df['quantity']
valve_df['amt_per_qty'] = valve_df['total_amt'] / valve_df['quantity']
valve_df['quantity_clipped'] = valve_df['quantity'].clip(upper=100)  # adjust as per your training
valve_df['last_bid_price_clipped'] = valve_df['last_bid_price'].clip(upper=100000)  # adjust as per your training
# brass_index_poly already calculated

# For EWM features, use the last value from training (simulate production)
last_ewm_proposed_rp = 35000  # <-- replace with your value
last_ewm_last_bid_price = 34000  # <-- replace with your value
valve_df['ewm_proposed_rp'] = last_ewm_proposed_rp
valve_df['ewm_last_bid_price'] = last_ewm_last_bid_price

# --- 4. Map copper and zinc prices using merge_asof ---
# Assume cu_price and zn_price are defined as dictionaries
cu_df = pd.DataFrame(list(cu_price.items()), columns=['date', 'cu_price'])
cu_df['date'] = pd.to_datetime(cu_df['date'])
zn_df = pd.DataFrame(list(zn_price.items()), columns=['date', 'zn_price'])
zn_df['date'] = pd.to_datetime(zn_df['date'])

valve_df = valve_df.sort_values('auction_date')
cu_df = cu_df.sort_values('date')
zn_df = zn_df.sort_values('date')

valve_df = pd.merge_asof(
    valve_df, cu_df, left_on='auction_date', right_on='date', direction='backward'
).drop(columns=['date'])
valve_df = pd.merge_asof(
    valve_df, zn_df, left_on='auction_date', right_on='date', direction='backward'
).drop(columns=['date'])

# --- 5. Predict brass_index_poly using saved model ---
brass_index_pipeline = joblib.load('../models/brass_index_model.joblib')
valve_df_renamed = valve_df.rename(columns={
    'cu_price': 'spot price(rs.)_copper',
    'zn_price': 'spot price(rs.)_zinc'
})

# Drop any rows with NaN copper or zinc prices (should be rare after merge_asof)
valve_df_renamed = valve_df_renamed.dropna(subset=['spot price(rs.)_copper', 'spot price(rs.)_zinc'])
valve_df = valve_df.loc[valve_df_renamed.index]

valve_df['brass_index_poly'] = brass_index_pipeline.predict(
    valve_df_renamed[['spot price(rs.)_copper', 'spot price(rs.)_zinc']]
)

# --- 6. Add missing feature if needed (e.g., 'state') ---
# If your model was trained with 'state', ensure it is present and in the same format
if 'state' not in valve_df.columns:
    valve_df['state'] = 'Unknown'  # or set to a default/most common value

# --- 7. Predict using the trained models ---
# Update this list to match the exact order and names used in training
valve_features = ['month', 'year', 'rp_per_qty', 'amt_per_qty', 'quantity_clipped', 'last_bid_price_clipped', 'brass_index_poly']
model_q25 = joblib.load('../models/valve_proposed_rp_q25.joblib')
model_q50 = joblib.load('../models/valve_proposed_rp_q50.joblib')
model_q75 = joblib.load('../models/valve_proposed_rp_q75.joblib')

X_pred = valve_df[valve_features]
valve_df['pred_q25'] = model_q25.predict(X_pred)
valve_df['pred_q50'] = model_q50.predict(X_pred)
valve_df['pred_q75'] = model_q75.predict(X_pred)

# --- 8. Compare predictions to actuals ---
valve_df['actual'] = valve_df['proposed_rp']
valve_df['abs_error'] = (valve_df['pred_q50'] - valve_df['actual']).abs()
valve_df['pct_error'] = 100 * valve_df['abs_error'] / valve_df['actual']
valve_df['in_range'] = valve_df.apply(
    lambda row: min(row['pred_q25'], row['pred_q75']) <= row['actual'] <= max(row['pred_q25'], row['pred_q75']),
    axis=1
)

# --- 9. Print summary metrics ---
mae = valve_df['abs_error'].mean()
rmse = np.sqrt(((valve_df['pred_q50'] - valve_df['actual']) ** 2).mean())
coverage = valve_df['in_range'].mean() * 100

print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"Coverage (actual in Q25-Q75): {coverage:.1f}%")

# --- 10. Save results to Excel ---
valve_df[['auction_date', 'quantity', 'actual', 'pred_q25', 'pred_q50', 'pred_q75', 'abs_error', 'pct_error', 'in_range']].to_excel('valve_rp_eval_results.xlsx', index=False)
print("Results saved to valve_rp_eval_results.xlsx") 

MAE: 12852.39
RMSE: 15392.43
Coverage (actual in Q25-Q75): 33.3%
Results saved to valve_rp_eval_results.xlsx
